In [1]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import copy
import numpy as np
from PIL import Image
from PIL import ImageFilter
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
drive.mount('/content/drive')
transform = transforms.Compose([transforms.Resize((256,256)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

Mounted at /content/drive


In [3]:
!unzip '/content/drive/MyDrive/Data.zip'

Archive:  /content/drive/MyDrive/Data.zip
   creating: Data/Test/
   creating: Data/Test/Mask/
  inflating: Data/Test/Mask/350.jpg  
  inflating: Data/Test/Mask/351.jpg  
  inflating: Data/Test/Mask/352.jpg  
  inflating: Data/Test/Mask/353.jpg  
  inflating: Data/Test/Mask/354.jpg  
  inflating: Data/Test/Mask/355.jpg  
  inflating: Data/Test/Mask/356.jpg  
  inflating: Data/Test/Mask/357.jpg  
  inflating: Data/Test/Mask/358.jpg  
  inflating: Data/Test/Mask/359.jpg  
  inflating: Data/Test/Mask/360.jpg  
  inflating: Data/Test/Mask/361.jpg  
  inflating: Data/Test/Mask/362.jpg  
  inflating: Data/Test/Mask/363.jpg  
  inflating: Data/Test/Mask/364.jpg  
  inflating: Data/Test/Mask/365.jpg  
  inflating: Data/Test/Mask/366.jpg  
  inflating: Data/Test/Mask/367.jpg  
  inflating: Data/Test/Mask/368.jpg  
  inflating: Data/Test/Mask/369.jpg  
  inflating: Data/Test/Mask/370.jpg  
  inflating: Data/Test/Mask/371.jpg  
  inflating: Data/Test/Mask/372.jpg  
  inflating: Data/Test/Mask/373

In [4]:
train_dataset = datasets.ImageFolder('/content/Data/Train',transform=transform)
validation_dataset = datasets.ImageFolder('/content/Data/Validation',transform=transform)
test_dataset = datasets.ImageFolder('/content/Data/Test',transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(16 * 128 * 128, 1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [6]:
#CENTRALIZED

In [7]:
model = CNN()
model = model.to(device)

In [8]:
optimizer = optim.Adam(model.parameters(),lr=0.005)
criterion = nn.BCEWithLogitsLoss()

In [9]:
# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        outputs = model(inputs)
        labels = labels.to(device)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in tqdm(validation_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            preds = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (preds == labels.float().view(-1, 1)).sum().item()
    accuracy = correct / total
    average_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

100%|██████████| 7/7 [00:00<00:00, 16.99it/s]


Epoch 1/20, Loss: 131.4462, Accuracy: 50.00%


100%|██████████| 7/7 [00:00<00:00, 16.84it/s]


Epoch 2/20, Loss: 210.2874, Accuracy: 50.00%


100%|██████████| 7/7 [00:00<00:00, 16.75it/s]


...

Epoch 18/20, Loss: 0.0462, Accuracy: 93.00%


100%|██████████| 7/7 [00:00<00:00, 14.35it/s]


Epoch 19/20, Loss: 0.0739, Accuracy: 95.00%


100%|██████████| 7/7 [00:00<00:00, 16.03it/s]

Epoch 20/20, Loss: 0.0393, Accuracy: 95.00%


In [10]:
# Test the models on the test_loader
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(validation_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (preds == labels.float().view(-1, 1)).sum().item()
    accuracy = correct /  total
print(f"Test Accuracy: {accuracy}")

100%|██████████| 7/7 [00:00<00:00, 11.84it/s]

Test Accuracy: 0.95


In [11]:
#FEDERATED

In [12]:
class Device():
    id = 0
    battery = 0
    communication_strength = 0
    def __init__(self, id, val1, val2):
        self.id = id
        self.battery = val1
        self.communication_strength = val2

clients = []
for i in range(0, 20):
    dev=Device(i, (100-i), 50)
    clients.append(dev)

In [13]:
model = CNN()
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=0.005)
criterion = nn.BCEWithLogitsLoss()

In [14]:
num_epochs = 5
temp_dataset = torch.utils.data.Subset(train_dataset, np.random.choice(len(train_dataset), 150, replace=True))
temp_loader = DataLoader(temp_dataset, batch_size=64, shuffle=False)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(temp_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        outputs = model(inputs)
        labels = labels.to(device)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    average_loss = running_loss / len(temp_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')

100%|██████████| 3/3 [00:00<00:00,  4.38it/s]


Epoch 1/5, Loss: 10.6458


100%|██████████| 3/3 [00:00<00:00,  3.79it/s]


Epoch 2/5, Loss: 0.9394


100%|██████████| 3/3 [00:00<00:00,  3.21it/s]


Epoch 3/5, Loss: 0.5902


100%|██████████| 3/3 [00:00<00:00,  3.23it/s]


Epoch 4/5, Loss: 0.4590


100%|██████████| 3/3 [00:00<00:00,  3.26it/s]

Epoch 5/5, Loss: 0.1087


In [15]:
model_weights = model.state_dict()

In [16]:
# Training Loop
num_rounds = 5
for round in range(num_rounds):
    for iter in range(20):
        bat = 0
        for client in clients:
            if(client.battery > bat):
                bat = client.battery
                dev = client
    dev.battery = dev.battery - 20
    temp_dataset = torch.utils.data.Subset(train_dataset, np.random.choice(len(train_dataset), 50, replace=True))
    temp_loader = DataLoader(temp_dataset, batch_size=64, shuffle=False)
    tmodel = copy.deepcopy(model)
    tmodel = tmodel.to(device)
    toptimizer = optim.Adam(model.parameters(),lr=0.005)
    tcriterion = nn.BCEWithLogitsLoss()
    for epoch in range(num_epochs):
        tmodel.train()
        running_loss = 0.0
        for inputs, labels in tqdm(temp_loader):
            toptimizer.zero_grad()
            inputs = inputs.to(device)
            outputs = tmodel(inputs)
            labels = labels.to(device)
            loss = tcriterion(outputs, labels.float().view(-1, 1))
            loss.backward()
            toptimizer.step()
            running_loss += loss.item()
            print(f'Round {round + 1}/{num_rounds}, Device {iter + 1}/20, Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')
    tmodel_weights = tmodel.state_dict()
    for key in model_weights:
        model_weights[key]=model_weights[key]+(tmodel_weights[key]-model_weights[key])/20
model.load_state_dict(model_weights)


100%|██████████| 1/1 [00:00<00:00,  3.18it/s]


Round 1/5, Device 20/20, Epoch 1/5, Loss: 0.1087


100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


Round 1/5, Device 20/20, Epoch 2/5, Loss: 0.1087


100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


...

100%|██████████| 1/1 [00:00<00:00,  3.33it/s]


Round 5/5, Device 20/20, Epoch 4/5, Loss: 0.1087


100%|██████████| 1/1 [00:00<00:00,  3.00it/s]

Round 5/5, Device 20/20, Epoch 5/5, Loss: 0.1087


<All keys matched successfully>

In [17]:
# Test the models on the test_loader
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(validation_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        preds = torch.round(torch.sigmoid(outputs))
        total += labels.size(0)
        correct += (preds == labels.float().view(-1, 1)).sum().item()
    accuracy = correct /  total
print(f"Test Accuracy: {accuracy}")

100%|██████████| 7/7 [00:00<00:00, 16.23it/s]

Test Accuracy: 0.9
